In [1]:
from tool import *

In [2]:
with open('D:/Data/boh_db.bin', 'rb') as f:
    boh_quanta = pickle.load(f)
boh_quanta

,Ref,Mapping Model.Suffix,BOH
0,2023-01-02(W01),11TG50Q-C.AC10KN,3
1,2023-01-02(W01),11TG50Q-E.AC10KN,54
2,2023-01-02(W01),15U50Q-G.AP5PL,144
3,2023-01-02(W01),15U50Q-G.AP70ML,4
4,2023-01-02(W01),15U50Q-G.AP75ML,30
...,...,...,...
40,2023-02-27(W09),16T90R-G.AA76K,5
41,2023-02-27(W09),16TD90R-G.AX56K,16
42,2023-02-27(W09),16TD90R-G.AX76K,35
43,2023-02-27(W09),16TG90Q-G.AA75KN,75


In [4]:
boh_ref_week = boh_quanta['Ref'].max()
boh_ref_week

'2023-02-27(W09)'

In [5]:
with open('D:/Data/Quanta Input Plan.bin', 'rb') as f:
    input_df = pickle.load(f)

In [6]:
input_df.head()

,Mapping Model.Suffix,Quanta P/N,Input Date,QTY,LG Week,Created_at
0,15UD50Q-G.AX50K,1LGHA000003,2022-04-29,150,2022-04-25(W17),2022-04-22
1,15U50Q-G.AR50K,1LGHA000004,2022-04-15,5,2022-04-11(W15),2022-04-22
2,15U50Q-G.AR50K,1LGHA000004,2022-04-20,70,2022-04-18(W16),2022-04-22
3,15U50Q-G.AR50K,1LGHA000004,2022-04-29,25,2022-04-25(W17),2022-04-22
4,15U50Q-G.AA56K,1LGHA000005,2022-04-29,50,2022-04-25(W17),2022-04-22


In [18]:
input_df[(input_df['LG Week'] == boh_ref_week) & (input_df['Created_at']==max_date)]

,Mapping Model.Suffix,Quanta P/N,Input Date,QTY,LG Week,Created_at
25005,16T90R-G.AA56K,1NLJC000008,2023-03-03,350,2023-02-27(W09),2023-03-31
25007,16T90R-G.AA5CK,1NLJC000009,2023-03-03,5,2023-02-27(W09),2023-03-31
25036,16T90Q-G.AA79K,1NLJA000090,2023-03-03,380,2023-02-27(W09),2023-03-31
25037,16TD90Q-G.AX70K,1NLJA000100,2023-03-03,80,2023-02-27(W09),2023-03-31
25040,16T90Q-G.AP76ML,1NLJA000118,2023-03-03,30,2023-02-27(W09),2023-03-31
25047,16T90Q-G.AA79B,1NLJA000148,2023-03-03,10,2023-02-27(W09),2023-03-31
25057,11TG50Q-E.AC10KN,1LI70000087,2023-03-01,290,2023-02-27(W09),2023-03-31


In [17]:
max_date = input_df[input_df['LG Week'] == boh_ref_week]['Created_at'].max()

In [ ]:
def get_DPK_stock():
    # DPK 재고의 가장 최신 재고의 산정 기준날짜 확인
    with open('D:/Data/DPK stock.bin', 'rb') as f:
        dpk_stock = pickle.load(f)

    # 재고 산정 기준 일자의 월부터 오늘날짜의 월까지의 Quanta의 월별 생산계획을 취합하기 위해 계산해야 할 월의 명단을 리스트로 만듬
    stock_ref_date = dpk_stock['Ref_date'].max().date()
    months = get_months_between_dates(stock_ref_date, datetime.date.today())

    # 취합할 생산계획 데이타프레임의 초기값을 빈 프레임으로 만들어 놓고 해당 월의 데이타를 누적시켜 생산계획을 취합함
    pr = pd.DataFrame()
    for target_month in months:
        if datetime.datetime.strftime(target_month, '%Y-%m') == datetime.datetime.strftime(stock_ref_date, '%Y-%m'):
            inp_plan = get_quanta_input_qty_by_month(target_month.year, target_month.month)
            inp_plan = inp_plan[inp_plan['Input Date'].dt.month == target_month.month]
            inp_plan = inp_plan[inp_plan['Input Date'] >= datetime.datetime.strftime(stock_ref_date, '%Y-%m-%d')]
            pr = pd.concat([pr, inp_plan]).reset_index(drop=True)
        else:
            inp_plan = get_quanta_input_qty_by_month(target_month.year, target_month.month)
            inp_plan = inp_plan[inp_plan['Input Date'].dt.month == target_month.month]
            pr = pd.concat([pr, inp_plan]).reset_index(drop=True)

    # 취합된 생산계획 데이타 프레임을 LG 주차별 생산계획 테이블로 전환하여  DPR의 OS 정보를 가져와 병합하여 OS별 생산계획으로 표현하고 여기에 DPK 기초 재고 정보도 추가함
    col = ['Model.Suffix', 'OS TYPE']
    pr_by_os = pr.pivot_table('QTY', index='Mapping Model.Suffix', columns='LG Week', aggfunc=sum).fillna(0).reset_index().merge(get_pdr()[col], left_on='Mapping Model.Suffix', right_on='Model.Suffix').drop(columns='Model.Suffix').groupby('OS TYPE').sum().reset_index()

    dpk_stock = pd.merge(pr_by_os, dpk_stock, how='outer').fillna(0).drop(columns='Ref_date')
    dpk_stock['MS P/N']= dpk_stock['OS TYPE'].map(dpk_map2)
    dpk_stock
    
    # DPK PO 정보를 만들어 둠
    with open('D:/Data/DPK blanket PO DB.bin', 'rb') as f:
            bpo = pickle.load(f)
    bpo = bpo.loc[bpo['LG PO Date'] > datetime.datetime.strftime(stock_ref_date, '%Y-%m-%d'), ('LG PO Date', 'MS P/N', 'Order Qty')].groupby(['LG PO Date', 'MS P/N']).sum()
    bpo = bpo.reset_index()
    bpo['LG Week'] = bpo['LG PO Date'].apply(get_weekname)
    bpo_pv = bpo.pivot_table('Order Qty', index='MS P/N', columns='LG Week')

    col1 = ['OS'] * 2 + get_weeklist(dpk_stock) * 3
    col2 = ['NAME', 'P/N'] + ['BOH'] * len(get_weeklist(dpk_stock)) + ['Input'] * len(get_weeklist(dpk_stock)) + ['DPK PO'] * len(get_weeklist(dpk_stock)) 
    
    # 각 주차별 생산/DPK 주문정보를 바탕으로 DPK 재고 정보를 계산함
    dpk_stock_balance = pd.DataFrame(data=0, index=range(len(dpk_stock['OS TYPE'])), columns=[col1, col2])
    dpk_stock_balance.loc[:, ('OS', 'NAME')] = dpk_stock['OS TYPE']
    dpk_stock_balance.loc[:, ('OS', 'P/N')] = dpk_stock['MS P/N']
    dpk_stock_balance.loc[:, (get_weeklist(dpk_stock)[0], 'BOH')] = dpk_stock['Stock']
    for week in get_weeklist(dpk_stock):
        dpk_stock_balance.loc[:, (week, 'Input')] = dpk_stock[week]
    for pn in bpo_pv.index:
        for week in bpo_pv.columns:
            c1 = (dpk_stock_balance[('OS', 'P/N')] == pn)
            dpk_stock_balance.loc[c1, (week, 'DPK PO')] = bpo_pv.loc[pn, week]
    dpk_stock_balance = dpk_stock_balance.fillna(0).convert_dtypes()

    for week in get_weeklist(dpk_stock)[1:]:
        for dpk in dpk_stock['MS P/N'].dropna():
            c1 = (dpk_stock_balance[('OS', 'P/N')] == dpk)
            dpk_stock_balance.loc[c1, (week, 'BOH')] = dpk_stock_balance.loc[c1, (get_weekname_from(week, -1) , 'BOH')].iloc[0] - dpk_stock_balance.loc[c1, (get_weekname_from(week, -1), 'Input')].iloc[0] + dpk_stock_balance.loc[c1, (get_weekname_from(week, -1), 'DPK PO')].iloc[0]
    return dpk_stock_balance.set_index([('OS', 'NAME'), ('OS', 'P/N')]).sort_index(axis=1)
